# **Create Synthetic Dataset**
In this notebook we will use the following structure to create a synthetic dataset of Intructions and Git Commands.

## **Install Dependencies**

In [ ]:
%pip install rich
%pip install openai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## **Import Library**

In [ ]:
from rich import print
import os
from openai import OpenAI
import json
import time
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## **Setting Up the API Key and Load Model llama 3.1**

In [2]:
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.environ["NVIDIA_API_KEY"]
)
MODEL = "meta/llama-3.1-405b-instruct"

## **Subtopics Generation**

In [3]:
n_subtopics = 3

TOPIC_GENERATION_PROMPT_TEMPLATE = """\
Saya ingin membuat dataset sintetis berupa instruksi dalam bahasa alami (Natural Language Instruction) dan respons berupa komentar media sosial. 
Berdasarkan konteks ini, berikan saya {n_subtopics} subtopik untuk mencakup berbagai jenis komentar yang bisa dibuat di media sosial.

List subtopik harus tanpa angka, dan tanpa deskripsi subtopik. Subtopik harus dipisahkan oleh koma. Tidak boleh ada teks lain selain daftar subtopik.
"""

def generate_subtopics(client, n_subtopics):
    prompt = TOPIC_GENERATION_PROMPT_TEMPLATE.format(n_subtopics=n_subtopics)
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0.2,
        top_p=0.7,
    )
    return response

responses = generate_subtopics(client, n_subtopics=n_subtopics)
print(responses.choices[0].message.content)

Komentar positif, komentar negatif, komentar netral

## **Instruction Generation**

In [4]:
n_instructions = 5

INSTRUCTION_PROMPT_TEMPLATE = """\
Tujuan kita adalah membuat dataset instruksi dari pengguna dalam bahasa sehari-hari (natural language instruction) yang nantinya akan direspons oleh AI pembuat komentar media sosial, tanpa menyebutkan spesifik jenis media sosialnya.
Berdasarkan topik yang diberikan, hasilkan {n_instructions} instruksi singkat yang mungkin diberikan kepada asisten AI untuk membuat komentar. Instruksi ini harus menyertakan jenis postingan yang akan dikomentari.

Beberapa instruksi harus ditulis seolah-olah diberikan diberikan oleh seseorang dengan pengetahuan terbatas tentang cara berkomentar, seperti pengguna pemula atau awam. 

Topiknya adalah: {sub_topic}
List instruksi harus tanpa nomor. Setiap instruksi harus dipisahkan oleh karakter baris baru. Tidak boleh ada teks lain selain list instruksi tersebut.
"""
subtopic_list = responses.choices[0].message.content.split(",")

def generate_instructions(client, sub_topic, n_instructions, retries=5, delay=5):
    print(f"Generate Instruksi untuk: [bold yellow]{sub_topic}[/bold yellow]")
    prompt = INSTRUCTION_PROMPT_TEMPLATE.format(sub_topic=sub_topic, n_instructions=n_instructions)
    for i in range(retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "user",
                     "content": prompt}
                ],
                temperature=0.2,
                top_p=0.7,
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            if "Too Many Requests" in str(e) and i < retries - 1:
                print(f"Rate limit exceeded. Retrying in {delay} seconds... (Attempt {i+1}/{retries})")
                time.sleep(delay)
                delay *= 2 
            else:
                raise e
    return "" 

def instructions_generator(client, subtopic_list, n_instructions):
    instruction_list = [generate_instructions(client, subtopic, n_instructions) for subtopic in subtopic_list]
    return instruction_list

instruction_list = instructions_generator(client, subtopic_list, n_instructions)

instruction_list_formatted = []
for instruction_set in instruction_list:
    instruction_list_formatted.extend([instruction.strip() for instruction in instruction_set.split("\n") if instruction])
print(instruction_list_formatted)

Generate Instruksi untuk: Komentar positif

Generate Instruksi untuk:  komentar negatif

Generate Instruksi untuk:  komentar netral

[
    'Tolong buat komentar positif untuk postingan foto teman saya yang baru saja melahirkan.',
    'Buat komentar yang menyenangkan untuk postingan video lucu tentang kucing.',
    'Saya ingin memberikan komentar yang baik untuk postingan artikel tentang kesehatan mental, tolong buatkan.',
    'Tolong buat komentar yang menyemangati untuk postingan foto seseorang yang baru saja lulus kuliah.',
    'Buat komentar yang sopan dan positif untuk postingan foto makanan yang lezat.',
    'Tolong buat komentar negatif untuk postingan foto makanan yang tidak enak.',
    'Buat komentar yang tidak menyukai postingan video musik baru yang kurang bagus.',
    'Saya ingin komentar yang mengekspresikan kekecewaan saya terhadap postingan berita politik yang tidak 
akurat.',
    'Tolong buat komentar singkat yang menunjukkan ketidaksetujuan saya terhadap postingan opini yang tidak masuk 
akal.',
    'Buat komentar yang mengekspresikan kesal saya terhadap postingan iklan yang menipu.',
    'Tolong buat komentar netral untuk postingan foto makanan yang lezat.',
    'Buat komentar yang tidak memihak untuk postingan berita politik.',
    'Saya ingin komentar yang sopan untuk postingan video lucu tentang hewan peliharaan.',
    'Tolong buat komentar singkat dan netral untuk postingan foto pemandangan alam yang indah.',
    'Buat komentar yang tidak menyinggung perasaan orang lain untuk postingan artikel tentang isu sosial.'
]

## **Input Generation**

In [5]:
INPUT_PROMPT_TEMPLATE = """\
Diberikan sebuah instruksi dari pengguna mengenai jenis komentar media sosial yang diinginkan, buatlah sebuah deskripsi tentang postingan media sosial yang relevan. Deskripsi ini akan berfungsi sebagai 'input' untuk AI pembuat komentar.
Fokus pada topik utama, suasana, atau kejadian yang ada di postingan, seolah-olah sedang menulis caption atau ringkasan konten postingan tersebut. Pastikan deskripsi realistis, beragam, dan bisa memicu komentar sesuai instruksi.

Instruksi: {instruction} 
Hasilkan hanya deskripsi itu sendiri. Tanpa teks pengantar, penutup, atau karakter tambahan di awal atau akhir deskripsi postingan.
"""

def generate_inputs(client, instruction, retries=5, delay=5):
    print(f"Generate Deskripsi Postingan untuk Instruksi: [bold yellow]{instruction}[/bold yellow]")
    prompt = INPUT_PROMPT_TEMPLATE.format(instruction=instruction)
    for i in range(retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "user",
                     "content": prompt}
                ],
                temperature=0.5, 
                top_p=0.8,       
                max_tokens=256,  
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            if "Too Many Requests" in str(e) and i < retries - 1:
                print(f"Rate limit exceeded. Retrying in {delay} seconds... (Attempt {i+1}/{retries})")
                time.sleep(delay)
                delay *= 2 
            else:
                raise e
    return "" 

def input_generator(client, instruction_list):
    input_list = []
    for instruction in instruction_list:
        input_list.append(generate_inputs(client, instruction))
    return input_list

input_description_list = input_generator(client, instruction_list_formatted)
input_description_pair_list = []
for instruction, post_description in zip(instruction_list_formatted, input_description_list):
    input_description_pair_list.append(
        {
            "instruction": instruction,
            "input": post_description,
        }
    )

print(input_description_pair_list)

Generate Deskripsi Postingan untuk Instruksi: Tolong buat komentar positif untuk postingan foto teman saya yang 
baru saja melahirkan.

Generate Deskripsi Postingan untuk Instruksi: Buat komentar yang menyenangkan untuk postingan video lucu tentang 
kucing.

Generate Deskripsi Postingan untuk Instruksi: Saya ingin memberikan komentar yang baik untuk postingan artikel 
tentang kesehatan mental, tolong buatkan.

Generate Deskripsi Postingan untuk Instruksi: Tolong buat komentar yang menyemangati untuk postingan foto seseorang
yang baru saja lulus kuliah.

Generate Deskripsi Postingan untuk Instruksi: Buat komentar yang sopan dan positif untuk postingan foto makanan 
yang lezat.

...

Rate limit exceeded. Retrying in 20 seconds... (Attempt 3/5)

Rate limit exceeded. Retrying in 40 seconds... (Attempt 4/5)

Generate Deskripsi Postingan untuk Instruksi: Tolong buat komentar singkat dan netral untuk postingan foto 
pemandangan alam yang indah.

Generate Deskripsi Postingan untuk Instruksi: Buat komentar yang tidak menyinggung perasaan orang lain untuk 
postingan artikel tentang isu sosial.

[
    {
        'instruction': 'Tolong buat komentar positif untuk postingan foto teman saya yang baru saja melahirkan.',
        'input': 'Foto bayi baru lahir yang lucu dan menggemaskan, dengan wajah mungil dan mata yang terpejam, 
diiringi dengan senyum bahagia dari sang ibu yang baru saja melahirkan.'
    },
    {
        'instruction': 'Buat komentar yang menyenangkan untuk postingan video lucu tentang kucing.',
        'input': 'Seekor kucing kecil berbulu putih sedang bermain dengan seutas benang, melompat-lompat dan 
menggelitiki dengan ekornya yang panjang dan lembut.'
    },
    {
        'instruction': 'Saya ingin memberikan komentar yang baik untuk postingan artikel tentang kesehatan mental, 
tolong buatkan.',
        'input': 'Artikel ini membahas tentang pentingnya menjaga kesehatan mental di tengah kesibukan sehari-hari.
Dengan gaya penulisan yang santai dan mudah dipahami, artikel ini menawarkan tips dan trik sederhana untuk 
mengurangi stres dan meningkatkan keseimbangan mental.'
    },
    {
        'instruction': 'Tolong buat komentar yang menyemangati untuk postingan foto seseorang yang baru saja lulus 
kuliah.',
        'input': 'Foto seorang muda berpakaian toga wisuda dengan senyum bangga dan tangan memegang ijazah, 
berlatar belakang gedung kampus yang megah.'
    },
    {
        'instruction': 'Buat komentar yang sopan dan positif untuk postingan foto makanan yang lezat.',
        'input': 'Foto makanan yang lezat ini menampilkan hidangan sushi yang segar dan menarik, dengan warna-warna
cerah dan tekstur yang menggugah selera. Sushi yang dihiasi dengan sayuran segar dan saus yang lezat ini tampak 
sangat menggugah selera dan membuat Anda ingin mencicipinya.'
    },
    {
        'instruction': 'Tolong buat komentar negatif untuk postingan foto makanan yang tidak enak.',
        'input': 'Foto makanan yang terlihat kurang menggugah selera dengan warna yang tidak menarik dan tekstur 
yang tidak mengundang.'
    },
    {
        'instruction': 'Buat komentar yang tidak menyukai postingan video musik baru yang kurang bagus.',
        'input': 'Video musik terbaru dari penyanyi populer yang baru saja dirilis, menampilkan gaya musik yang 
berbeda dari biasanya dengan lirik yang kurang bermakna dan aransemen yang kurang menarik.'
    },
    {
        'instruction': 'Saya ingin komentar yang mengekspresikan kekecewaan saya terhadap postingan berita politik 
yang tidak akurat.',
        'input': 'Sebuah postingan berita politik yang memuat informasi tidak akurat tentang kinerja pemerintah 
saat ini, menyebutkan bahwa pemerintah telah berhasil menyelesaikan proyek infrastruktur besar dalam waktu singkat,
padahal faktanya proyek tersebut masih terhambat dan belum selesai.'
    },
    {
        'instruction': 'Tolong buat komentar singkat yang menunjukkan ketidaksetujuan saya terhadap postingan opini
yang tidak masuk akal.',
        'input': 'Seorang influencer terkenal membagikan pendapatnya bahwa makanan cepat saji lebih sehat daripada 
makanan rumahan, dengan alasan bahwa makanan cepat saji telah melalui proses pengolahan yang lebih ketat dan 
terjamin kebersihannya. Postingan tersebut disertai dengan foto-foto makanan cepat saji yang menarik dan dikemas 
dengan baik, serta beberapa kalimat yang menekankan pada kenyamanan dan kepraktisan makanan cepat saji.'
    },
    {
        'instruction': 'Buat komentar yang mengekspresikan kesal saya terhadap postingan iklan yang menipu.',
        'input': 'Iklan produk kecantikan yang menjanjikan hasil instan dan tidak masuk akal, dengan foto-foto 
model yang terlalu sempurna dan tidak realistis, serta testimonial yang terdengar dibuat-buat.'
    },
    {
        'instruction': 'Tolong buat komentar netral untuk postingan foto makanan yang lezat.',
        'input': 'Foto makanan yang lezat dengan warna-warna cerah dan aroma yang menggugah selera. Hidangan utama 
adalah daging panggang yang disajikan dengan sayuran segar dan saus khas. Penyajian yang rapi dan menarik mem

## **Output Generation**

In [6]:
OUTPUT_PROMPT_TEMPLATE = """\
Anda adalah AI pembuat komentar media sosial yang ahli dalam menulis komentar yang relevan, natural, dan sesuai konteks.
Diberikan sebuah instruksi dari pengguna dan deskripsi postingan media sosial, buatlah sebuah komentar yang realistis dan alami seolah-olah ditulis oleh manusia.

Gunakan bahasa sehari-hari atau bahasa gaul yang umum di media sosial (sesuai konteks instruksi). Komentar harus singkat, tidak terlalu formal, dan bisa mencakup emoji atau singkatan yang wajar jika sesuai. Hindari pengulangan dan pastikan komentar tidak terdengar seperti robot.

Instruksi: {instruction}
Deskripsi Postingan: {input_description}

Hasilkan hanya komentar itu sendiri, tanpa teks pengantar, penutup, atau karakter tambahan di awal atau akhir komentar.
"""

def generate_outputs(client, instruction, input_description, retries=5, delay=5):
    print(f"Generate Komentar untuk Instruksi: [bold yellow]{instruction}[/bold yellow] dan Postingan: [bold cyan]{input_description[:50]}...[/bold cyan]")
    prompt = OUTPUT_PROMPT_TEMPLATE.format(instruction=instruction, input_description=input_description)
    for i in range(retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "user",
                     "content": prompt}
                ],
                temperature=0.7, 
                top_p=0.8,      
                max_tokens=100,  
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            if "Too Many Requests" in str(e) and i < retries - 1:
                print(f"Rate limit exceeded. Retrying in {delay} seconds... (Attempt {i+1}/{retries})")
                time.sleep(delay) 
                delay *= 2 
            else:
                raise e
    return "" 

def output_generator(client, instruction_input_list):
    generated_outputs = [] 
    for item in instruction_input_list:
        comment = generate_outputs(client, item['instruction'], item['input'])
        generated_outputs.append(
            {
                "instruction": item['instruction'],
                "input": item['input'],
                "output": comment
            }
        )
    return generated_outputs 

# Panggil fungsi output_generator dan simpan hasilnya ke output_list
output_list = output_generator(client, input_description_pair_list)

print(output_list)

Generate Komentar untuk Instruksi: Tolong buat komentar positif untuk postingan foto teman saya yang baru saja 
melahirkan. dan Postingan: Foto bayi baru lahir yang lucu dan menggemaskan, d...

Generate Komentar untuk Instruksi: Buat komentar yang menyenangkan untuk postingan video lucu tentang kucing. dan 
Postingan: Seekor kucing kecil berbulu putih sedang bermain d...

Generate Komentar untuk Instruksi: Saya ingin memberikan komentar yang baik untuk postingan artikel tentang 
kesehatan mental, tolong buatkan. dan Postingan: Artikel ini membahas tentang pentingnya menjaga ke...

Generate Komentar untuk Instruksi: Tolong buat komentar yang menyemangati untuk postingan foto seseorang yang baru 
saja lulus kuliah. dan Postingan: Foto seorang muda berpakaian toga wisuda dengan se...

Generate Komentar untuk Instruksi: Buat komentar yang sopan dan positif untuk postingan foto makanan yang lezat. 
dan Postingan: Foto makanan yang lezat ini menampilkan hidangan s...

...

Rate limit exceeded. Retrying in 5 seconds... (Attempt 1/5)

Rate limit exceeded. Retrying in 10 seconds... (Attempt 2/5)

Rate limit exceeded. Retrying in 20 seconds... (Attempt 3/5)

Generate Komentar untuk Instruksi: Buat komentar yang tidak menyinggung perasaan orang lain untuk postingan artikel
tentang isu sosial. dan Postingan: Artikel ini membahas tentang isu sosial yang sedan...

[
    {
        'instruction': 'Tolong buat komentar positif untuk postingan foto teman saya yang baru saja melahirkan.',
        'input': 'Foto bayi baru lahir yang lucu dan menggemaskan, dengan wajah mungil dan mata yang terpejam, 
diiringi dengan senyum bahagia dari sang ibu yang baru saja melahirkan.',
        'output': 'Selamat ya! Bayinya lucu banget, mirip sama kamu! 😊 Semoga sehat selalu dan bahagia bersama 
keluarga baru. 💕'
    },
    {
        'instruction': 'Buat komentar yang menyenangkan untuk postingan video lucu tentang kucing.',
        'input': 'Seekor kucing kecil berbulu putih sedang bermain dengan seutas benang, melompat-lompat dan 
menggelitiki dengan ekornya yang panjang dan lembut.',
        'output': 'Hahaaha kucingnya lucu banget! 😂 kayaknya dia lagi latihan jadi penari balet 🐱💃'
    },
    {
        'instruction': 'Saya ingin memberikan komentar yang baik untuk postingan artikel tentang kesehatan mental, 
tolong buatkan.',
        'input': 'Artikel ini membahas tentang pentingnya menjaga kesehatan mental di tengah kesibukan sehari-hari.
Dengan gaya penulisan yang santai dan mudah dipahami, artikel ini menawarkan tips dan trik sederhana untuk 
mengurangi stres dan meningkatkan keseimbangan mental.',
        'output': 'Artikel yang sangat bermanfaat! Terima kasih sudah berbagi tips dan trik untuk menjaga kesehatan
mental. Sangat penting untuk kita semua ingat bahwa kesehatan mental sama pentingnya dengan kesehatan fisik. Mari 
kita jaga kesehatan mental kita dengan baik! 💆\u200d♀️🙏'
    },
    {
        'instruction': 'Tolong buat komentar yang menyemangati untuk postingan foto seseorang yang baru saja lulus 
kuliah.',
        'input': 'Foto seorang muda berpakaian toga wisuda dengan senyum bangga dan tangan memegang ijazah, 
berlatar belakang gedung kampus yang megah.',
        'output': 'Selamat ya! Akhirnya kelar juga! Bangga banget sama kamu yang udah berhasil melewati semua 
tantangan di kampus. Semoga ilmunya bermanfaat dan masa depanmu cerah! 🎓💫'
    },
    {
        'instruction': 'Buat komentar yang sopan dan positif untuk postingan foto makanan yang lezat.',
        'input': 'Foto makanan yang lezat ini menampilkan hidangan sushi yang segar dan menarik, dengan warna-warna
cerah dan tekstur yang menggugah selera. Sushi yang dihiasi dengan sayuran segar dan saus yang lezat ini tampak 
sangat menggugah selera dan membuat Anda ingin mencicipinya.',
        'output': '"Wah, sushi ini keliatannya lezat banget! Warnanya cerah, teksturnya pas, dan hiasannya keren. 
Jadi pengen nyobain!" 🤤🍣'
    },
    {
        'instruction': 'Tolong buat komentar negatif untuk postingan foto makanan yang tidak enak.',
        'input': 'Foto makanan yang terlihat kurang menggugah selera dengan warna yang tidak menarik dan tekstur 
yang tidak mengundang.',
        'output': '"Gak enak banget kayaknya, warnanya aja udah gak menggugah selera. Teksturnya juga kayaknya 
keras gitu"'
    },
    {
        'instruction': 'Buat komentar yang tidak menyukai postingan video musik baru yang kurang bagus.',
        'input': 'Video musik terbaru dari penyanyi populer yang baru saja dirilis, menampilkan gaya musik yang 
berbeda dari biasanya dengan lirik yang kurang bermakna dan aransemen yang kurang menarik.',
        'output': '"Menurutku, lagu ini kurang nendang dibandingkan sama lagu-lagu sebelumnya. Liriknya kurang 
bermakna dan aransemennya kurang menarik. Semoga kembali ke gaya musik yang dulu lagi." 🤔'
    },
    {
        'instruction': 'Saya ingin komentar yang mengekspresikan kekecewaan saya terhadap postingan berita politik 
yang tidak akurat.',
        'input': 'Sebuah postingan berita politik yang memuat informasi tidak akurat tentang kinerja pemerintah 
saat ini, menyebutkan bahwa pemerintah telah berhasil menyelesaikan proyek infrastruktur besar dalam waktu singkat,
padahal faktanya proyek tersebut masih terhambat dan belum selesai.',
        'output': '"Kok bisa sih beritanya nggak akurat gini? Proyek infrastruktur

## **Saving the Dataset**

In [7]:
output_filename = 'dataset_sosmed2.jsonl'
with open(output_filename, 'w', encoding='utf-8') as f:
    for item in output_list:
        f.write(json.dumps(item))
        f.write('\n')
print(f"\nDataset berhasil disimpan ke: [bold magenta]{output_filename}[/bold magenta]")

Dataset berhasil disimpan ke: dataset_sosmed2.jsonl

In [8]:
import csv
output_filename = 'dataset_sosmed2.csv'
fieldnames = output_list[0].keys()  # Asumsikan semua elemen memiliki struktur yang sama

with open(output_filename, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()  # Menulis header
    writer.writerows(output_list)  # Menulis data

print(f"\nDataset berhasil disimpan ke: [bold magenta]{output_filename}[/bold magenta]")

Dataset berhasil disimpan ke: dataset_sosmed2.csv